# Extract Image

In [1]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('/home/srv/Work/PythonScript/distance_calculation/image/IMG_6893.jpg')

# Detect the markers in the image
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
aruco_params = cv2.aruco.DetectorParameters()
corners, ids, rejected = cv2.aruco.detectMarkers(image, aruco_dict, parameters=aruco_params)

# Verify that 4 markers are detected
print(ids)
if ids is not None and len(ids) == 4:
    # Flatten the ids list
    ids = ids.flatten()

    # Dictionary to store the marker id and its corresponding corners
    marker_corners = {}
    
    for marker_corner, marker_id in zip(corners, ids):
        # Get the corner points of each marker
        marker_corners[marker_id] = marker_corner[0]

    # Sort the marker ids to maintain a consistent order
    sorted_ids = sorted(marker_corners.keys())

    # Get the corners of the markers in order
    pts = np.array([marker_corners[sorted_ids[0]][0], marker_corners[sorted_ids[1]][1],
                    marker_corners[sorted_ids[2]][2], marker_corners[sorted_ids[3]][3]], dtype='float32')

    # Create a blank 990mm x 990mm image
    dpi = 300  # Assuming the DPI (dots per inch) is 300
    mm_per_inch = 25.4
    pixels_per_mm = dpi / mm_per_inch
    size_in_pixels = int(990 * pixels_per_mm)
    blank_image = np.zeros((size_in_pixels, size_in_pixels, 3), dtype=np.uint8)

    # Define the destination points (corners of the blank image)
    dst_pts = np.array([[0, 0], [size_in_pixels - 1, 0], [size_in_pixels - 1, size_in_pixels - 1], [0, size_in_pixels - 1]], dtype='float32')

    # Get the actual marker sizes and add padding if necessary
    marker_size = max(np.linalg.norm(pts[0] - pts[1]), np.linalg.norm(pts[1] - pts[2]))
    padding =  int(marker_size / 8)

    # Adjust the destination points to ensure markers fit well within the corners with padding
    dst_pts[0] += [padding, padding]
    dst_pts[1] += [-padding, padding]
    dst_pts[2] += [-padding, -padding]
    dst_pts[3] += [padding, -padding]

    # Compute the perspective transform matrix
    M = cv2.getPerspectiveTransform(pts, dst_pts)

    # Warp the original image to the new image size
    warped = cv2.warpPerspective(image, M, (size_in_pixels, size_in_pixels))

    # Save the result
    cv2.imwrite("990mm_x_990mm_Image.jpg", warped)
    # cv2.imshow("990mm x 990mm Image", warped)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
else:
    print("Could not detect 4 markers.")


[[2]
 [1]
 [0]
 [3]]


# Apply grid

In [2]:
import numpy as np
import cv2

# Define the grid sizes along the x and y axes
x_grid_sizes = [120, 190, 190, 190, 180, 120]
y_grid_sizes = [120, 190, 190, 190, 180, 120]

# Calculate the number of grid points
x_points = len(x_grid_sizes) + 1
y_points = len(y_grid_sizes) + 1

# Initialize the image
dpi = 300  # Assuming the DPI (dots per inch) is 300
mm_per_inch = 25.4
pixels_per_mm = dpi / mm_per_inch
image_size = int(990 * pixels_per_mm)
image = warped.copy()  # Use the previously warped image

# Calculate the pixel positions for the grid lines
x_positions = np.cumsum([0] + x_grid_sizes) * pixels_per_mm
y_positions = np.cumsum([0] + y_grid_sizes) * pixels_per_mm

# Define line and circle properties
line_color = (0, 255, 0)  # Green color for grid lines
line_thickness = 3  # Thickness of the grid lines
circle_color = (0, 0, 255)  # Red color for circles
circle_radius = 10  # Radius of the circles
circle_thickness = 2  # Thickness of the circle edges

# Draw the grid lines
for x in x_positions:
    cv2.line(image, (int(x), 0), (int(x), image_size), line_color, line_thickness)
for y in y_positions:
    cv2.line(image, (0, int(y)), (image_size, int(y)), line_color, line_thickness)

# Define text properties
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 3
font_color = (255, 255, 255)  # White color for text
font_thickness = 5

# Print the x, y values of each matching point and mark them on the image
for i, x in enumerate(x_positions):
    for j, y in enumerate(y_positions):
        x_mm = x / pixels_per_mm
        # y_mm = (image_size - y) / pixels_per_mm  # Adjust to start y from the bottom
        y_mm = y/pixels_per_mm
        text = f'x={x_mm:.1f}mm, y={round(y_mm):.1f}mm'
        print(text)
        cv2.circle(image, (int(x), int(y)), circle_radius, circle_color, circle_thickness)
        cv2.putText(image, text, (int(x) + 10, int(y) - 10), font, font_scale, font_color, font_thickness)

# Save and display the result
cv2.imwrite("990mm_x_990mm_grid.jpg", image)
# cv2.imshow("990mm x 990mm Grid", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


x=0.0mm, y=0.0mm
x=0.0mm, y=120.0mm
x=0.0mm, y=310.0mm
x=0.0mm, y=500.0mm
x=0.0mm, y=690.0mm
x=0.0mm, y=870.0mm
x=0.0mm, y=990.0mm
x=120.0mm, y=0.0mm
x=120.0mm, y=120.0mm
x=120.0mm, y=310.0mm
x=120.0mm, y=500.0mm
x=120.0mm, y=690.0mm
x=120.0mm, y=870.0mm
x=120.0mm, y=990.0mm
x=310.0mm, y=0.0mm
x=310.0mm, y=120.0mm
x=310.0mm, y=310.0mm
x=310.0mm, y=500.0mm
x=310.0mm, y=690.0mm
x=310.0mm, y=870.0mm
x=310.0mm, y=990.0mm
x=500.0mm, y=0.0mm
x=500.0mm, y=120.0mm
x=500.0mm, y=310.0mm
x=500.0mm, y=500.0mm
x=500.0mm, y=690.0mm
x=500.0mm, y=870.0mm
x=500.0mm, y=990.0mm
x=690.0mm, y=0.0mm
x=690.0mm, y=120.0mm
x=690.0mm, y=310.0mm
x=690.0mm, y=500.0mm
x=690.0mm, y=690.0mm
x=690.0mm, y=870.0mm
x=690.0mm, y=990.0mm
x=870.0mm, y=0.0mm
x=870.0mm, y=120.0mm
x=870.0mm, y=310.0mm
x=870.0mm, y=500.0mm
x=870.0mm, y=690.0mm
x=870.0mm, y=870.0mm
x=870.0mm, y=990.0mm
x=990.0mm, y=0.0mm
x=990.0mm, y=120.0mm
x=990.0mm, y=310.0mm
x=990.0mm, y=500.0mm
x=990.0mm, y=690.0mm
x=990.0mm, y=870.0mm
x=990.0mm, y=990.0mm

True

# detect circle

In [13]:
import cv2
import numpy as np

# Load the image

image = warped.copy()
output = image.copy()

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply GaussianBlur to reduce noise and improve circle detection
gray_blurred = cv2.GaussianBlur(gray, (15, 15), 1)

# Apply Hough Circle Transform to detect circles
detected_circles = cv2.HoughCircles(gray_blurred, 
                                    cv2.HOUGH_GRADIENT,
                                    dp=1.33,
                                    minDist=1700,
                                    param1=48,
                                    param2=30, 
                                    minRadius=200, 
                                    maxRadius=210)

# If circles are detected, draw them on the output image
circle_centers = []
if detected_circles is not None:
    detected_circles = np.uint16(np.around(detected_circles))
    for (x, y, r) in detected_circles[0, :]:
        # y = (image_size - y)
        circle_centers.append((x, y, r))
        cv2.circle(output, (x, y), r, (0, 255, 0), 10)
        cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 135, 255), -1)

# Convert circle centers and radius to mm
dpi = 300  # Assuming the DPI (dots per inch) is 300
mm_per_inch = 25.4
pixels_per_mm = dpi / mm_per_inch

circle_centers_mm = []
for (x, y, r) in circle_centers:
    x_mm = x / pixels_per_mm
    y_mm = y / pixels_per_mm
    r_mm = r / pixels_per_mm
    circle_centers_mm.append((x_mm, y_mm, r_mm))
    print(f'Circle center in mm: x={x_mm:.2f}, y={y_mm:.2f}, radius={r_mm:.2f}')
    
    # Plot the center coordinates in mm on the image
    text = f'({x_mm:.2f}mm, {y_mm:.2f}mm)'
    cv2.putText(output, text, (x + 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

# Save the output image with detected circles
cv2.imwrite("detect_with_mm.jpg", output)



Circle center in mm: x=867.83, y=298.79, radius=17.10
Circle center in mm: x=868.93, y=868.17, radius=16.93
Circle center in mm: x=311.32, y=870.71, radius=17.19
Circle center in mm: x=688.85, y=119.72, radius=17.10
Circle center in mm: x=499.03, y=119.63, radius=17.10
Circle center in mm: x=689.02, y=868.85, radius=16.93
Circle center in mm: x=868.93, y=489.20, radius=16.93
Circle center in mm: x=119.21, y=299.72, radius=17.19
Circle center in mm: x=867.49, y=118.28, radius=17.19
Circle center in mm: x=308.61, y=121.67, radius=17.10
Circle center in mm: x=869.95, y=678.86, radius=17.19
Circle center in mm: x=192.53, y=767.16, radius=17.70
Circle center in mm: x=119.89, y=120.90, radius=17.19
Circle center in mm: x=505.29, y=869.27, radius=17.10
Circle center in mm: x=120.73, y=494.88, radius=17.19


True

# final plotting 

In [11]:
image = warped.copy()
for i, x in enumerate(x_positions):
    for j, y in enumerate(y_positions):
        x_mm = x / pixels_per_mm
        # y_mm = (image_size - y) / pixels_per_mm  # Adjust to start y from the bottom
        y_mm = y/pixels_per_mm
        text = f'x={x_mm:.1f}mm, y={round(y_mm):.1f}mm'
        print(text)
        cv2.circle(image, (int(x), int(y)), circle_radius, circle_color, circle_thickness)
        cv2.putText(image, text, (int(x) + 10, int(y) - 10), font, font_scale, font_color, font_thickness)

for (x, y, r) in circle_centers:
    x_mm = x / pixels_per_mm
    y_mm = y / pixels_per_mm
    r_mm = r / pixels_per_mm
    circle_centers_mm.append((x_mm, y_mm, r_mm))
    print(f'Circle center in mm: x={x_mm:.2f}, y={y_mm:.2f}, radius={r_mm:.2f}')
    
    # Plot the center coordinates in mm on the image
    text = f'({x_mm:.2f}mm, {y_mm:.2f}mm)'
    cv2.putText(image, text, (x + 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 10, (255, 0, 0), 2)

# Save the output image with detected circles
cv2.imwrite("final_mm.jpg", image)
# Save and display the result

x=0.0mm, y=0.0mm
x=0.0mm, y=120.0mm
x=0.0mm, y=310.0mm
x=0.0mm, y=500.0mm
x=0.0mm, y=690.0mm
x=0.0mm, y=870.0mm
x=0.0mm, y=990.0mm
x=120.0mm, y=0.0mm
x=120.0mm, y=120.0mm
x=120.0mm, y=310.0mm
x=120.0mm, y=500.0mm
x=120.0mm, y=690.0mm
x=120.0mm, y=870.0mm
x=120.0mm, y=990.0mm
x=310.0mm, y=0.0mm
x=310.0mm, y=120.0mm
x=310.0mm, y=310.0mm
x=310.0mm, y=500.0mm
x=310.0mm, y=690.0mm
x=310.0mm, y=870.0mm
x=310.0mm, y=990.0mm
x=500.0mm, y=0.0mm
x=500.0mm, y=120.0mm
x=500.0mm, y=310.0mm
x=500.0mm, y=500.0mm
x=500.0mm, y=690.0mm
x=500.0mm, y=870.0mm
x=500.0mm, y=990.0mm
x=690.0mm, y=0.0mm
x=690.0mm, y=120.0mm
x=690.0mm, y=310.0mm
x=690.0mm, y=500.0mm
x=690.0mm, y=690.0mm
x=690.0mm, y=870.0mm
x=690.0mm, y=990.0mm
x=870.0mm, y=0.0mm
x=870.0mm, y=120.0mm
x=870.0mm, y=310.0mm
x=870.0mm, y=500.0mm
x=870.0mm, y=690.0mm
x=870.0mm, y=870.0mm
x=870.0mm, y=990.0mm
x=990.0mm, y=0.0mm
x=990.0mm, y=120.0mm
x=990.0mm, y=310.0mm
x=990.0mm, y=500.0mm
x=990.0mm, y=690.0mm
x=990.0mm, y=870.0mm
x=990.0mm, y=990.0mm

True

In [13]:
image = warped.copy()

cicle_dict = {}

distances = []
for (x_c, y_c, r_c) in circle_centers:
    x_mm_circle = x_c / pixels_per_mm
    y_mm_circle = y_c / pixels_per_mm
    r_mm = r / pixels_per_mm
    grids = []

    for i, x_g in enumerate(x_positions):
        for j, y_g in enumerate(y_positions):
            x_mm_grid = x_g / pixels_per_mm
            # y_mm = (image_size - y) / pixels_per_mm  # Adjust to start y from the bottom
            y_mm_grid = y_g/pixels_per_mm
            distance = np.sqrt((x_mm_circle - x_mm_grid)**2 + (y_mm_circle - y_mm_grid)**2)
            grids.append({(int(x_g), int(y_g)):distance})
#     new_dict = {"distance": distances, "grids": grids}
    cicle_dict[(int(x_c), int(y_c))]  = grids


In [14]:
image = warped.copy()
for key_c in  cicle_dict.keys():
    min = 9999999
    grid_ = key_c
    # print(cicle_dict[key_c])
    
    for key_g in cicle_dict[key_c]:
        key =list(key_g.keys())
        dis = key_g[key[0]]
        # print(dis)
        if dis<min:
            min =dis
            grid_ = key[0]
    cv2.circle(image, key_c, circle_radius, (255,255,255), circle_thickness)
    cv2.circle(image, grid_, circle_radius, (0,0,255), circle_thickness)
    
    # cv2.circle(image, grid_, circle_radius, grid_circle_color, circle_thickness)
    image = cv2.line(image, key_c,grid_ ,(0,255,0),9)
    print("center", key_c)
    print("grid", grid_)
    print("distance", dis)
    print("--------------------")
    cv2.imwrite("distance_mm.jpg", image)

center (10266, 10253)
grid (10275, 10275)
distance 171.63402237836704
--------------------
center (10249, 3526)
grid (10275, 3661)
distance 702.1892164534823
--------------------
center (8137, 10261)
grid (8149, 10275)
distance 324.56054173125995
--------------------
center (3679, 10282)
grid (3661, 10275)
distance 688.9467932641018
--------------------
center (8138, 1412)
grid (8149, 1417)
distance 921.0184193240774
--------------------
center (5894, 1413)
grid (5905, 1417)
distance 999.2963010361063
--------------------
center (3648, 1436)
grid (3661, 1417)
distance 1103.6744226043797
--------------------
center (10276, 8019)
grid (10275, 8149)
distance 333.3898027321438
--------------------
center (1412, 5793)
grid (1417, 5905)
distance 1003.5988181422118
--------------------
center (10246, 1398)
grid (10275, 1417)
distance 880.2027455030523
--------------------
center (1408, 3540)
grid (1417, 3661)
distance 1111.1977958253478
--------------------
center (10264, 5775)
grid (10275, 5

In [2]:
from ultralytics import YOLO

In [57]:
model = YOLO("source/weights/best.pt")
img = cv2.imread("/home/srv/Work/PythonScript/distance_calculation/source/result/extracted_image.jpg")
results = model(img)
# print(results)
# # Process results list
# for result in results:
#     boxes = result.boxes  # Boxes object for bounding box outputs
#     masks = result.masks  # Masks object for segmentation masks outputs
#     keypoints = result.keypoints  # Keypoints object for pose outputs
#     probs = result.probs  # Probs object for classification outputs
#     # obb = result.obb  # Oriented boxes object for OBB outputs
#     # result.show()  # display to screen
#     # result.save(filename="result.jpg")  # save to disk


0: 640x640 16 circles, 3905.3ms
Speed: 26.2ms preprocess, 3905.3ms inference, 37.4ms postprocess per image at shape (1, 3, 640, 640)


In [48]:
# model = YOLO("/home/srv/Work/PythonScript/distance_calculation/best.pt")
# results = model("/home/srv/Work/PythonScript/distance_calculation/source/result/extracted_image.jpg")
# # Process results list
# for result in results:
#     boxes = result.boxes  # Boxes object for bounding box outputs
#     masks = result.masks  # Masks object for segmentation masks outputs
#     keypoints = result.keypoints  # Keypoints object for pose outputs
#     probs = result.probs  # Probs object for classification outputs
#     # obb = result.obb  # Oriented boxes object for OBB outputs
#     # result.show()  # display to screen
#     # result.save(filename="result.jpg")  # save to disk


image 1/1 /home/srv/Work/PythonScript/distance_calculation/source/result/extracted_image.jpg: 640x640 16 circles, 3695.5ms
Speed: 7.7ms preprocess, 3695.5ms inference, 39.5ms postprocess per image at shape (1, 3, 640, 640)


In [56]:
boxes = results[0].boxes.xyxy
img = cv2.imread("/home/srv/Work/PythonScript/distance_calculation/source/result/extracted_image.jpg")

for box in boxes:
    x_min,y_min,x_max,y_max = box
    x_min = int(x_min)
    y_min = int(y_min)
    x_max = int(x_max)
    y_max = int(y_max)
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    radius = int(min(x_max - x_min, y_max - y_min) / 2)
    cv2.circle(img, (int(x_center), int(y_center)), radius, (0, 255, 0), 10)  # Green color, 2px thickness

    # Draw the bounding box on the image
    # cv2.rectangle(img,(x_min,y_min),(x_max,y_max), (0,255,0), 10)
    cv2.circle(img, (int(x_center), int(y_center)), radius=10, color=(0, 0, 255), thickness=5)  # Red dot
    

cv2.imwrite("output_.jpg", img)

True

In [76]:
a=  -0.16933333333332712
x = 0 if a >=-1 and a < 1 else 1
x

0

1